<a href="https://colab.research.google.com/github/Sumitkevlani/LLM/blob/main/Agents_In_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install --upgrade --quiet langchain langchain-core langchain-huggingface langchain-openai langchain-community wikipedia

In [31]:
from huggingface_hub import login
login()

In [32]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint

In [33]:
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    max_new_tokens=64,
    do_sample=False,
    repetition_penalty=1.03,
)

In [34]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain_community import tools
from langchain_community import utilities

In [35]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [36]:
from google.colab import userdata
tavily_api_key = userdata.get('TAVILY_API_KEY')

In [37]:
import os
os.environ['TAVILY_API_KEY'] = tavily_api_key

In [38]:
search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Pune")
print(search_results)

[{'url': 'https://world-weather.info/forecast/india/pune/june-2024/', 'content': 'Extended weather forecast in Pune. Hourly Week 10 days 14 days 30 days Year. Detailed ⚡ Pune Weather Forecast for June 2024 - day/night 🌡️ temperatures, precipitations - World-Weather.info.'}, {'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Pune', 'region': 'Maharashtra', 'country': 'India', 'lat': 18.53, 'lon': 73.87, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1719439196, 'localtime': '2024-06-27 3:29'}, 'current': {'last_updated_epoch': 1719438300, 'last_updated': '2024-06-27 03:15', 'temp_c': 24.7, 'temp_f': 76.4, 'is_day': 0, 'condition': {'text': 'Light rain shower', 'icon': '//cdn.weatherapi.com/weather/64x64/night/353.png', 'code': 1240}, 'wind_mph': 6.0, 'wind_kph': 9.7, 'wind_degree': 241, 'wind_dir': 'WSW', 'pressure_mb': 1001.0, 'pressure_in': 29.56, 'precip_mm': 0.27, 'precip_in': 0.01, 'humidity': 89, 'cloud': 100, 'feelslike_c': 27.2, 'feelslike_f': 81.0, 'windchil

In [39]:
tools = [search]

In [40]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [41]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [42]:
wikipedia.run("How many years ago Jallianwala Bagh Massacre happened?")

'Page: Jallianwala Bagh massacre\nSummary: The Jallianwala Bagh massacre, also known as the Amritsar massacre, took place on 13 April 1919. A large, peaceful crowd had gathered at the Jallianwala Bagh in Amritsar, Punjab, British India, during the annual Baishakhi fair to protest against the Rowlatt Act and the arrest of pro-independence activists Saifuddin Kitchlew and Satyapal. In response to the public gathering, the temporary brigadier general R. E. H. Dyer surrounded the people with his Gurkha and Sikh infantry regiments of the British Indian Army. The Jallianwala Bagh could only be exited on one side, as its other three sides were enclosed by buildings. After blocking the exit with his troops, Dyer ordered them to shoot at the crowd, continuing to fire even as the protestors tried to flee. The troops kept on firing until their ammunition was exhausted. Estimates of those killed vary from 379 to 1,500 or more people; over 1,200 others were injured, of whom 192 sustained serious in

In [44]:
from langchain_community.utilities import SearchApiAPIWrapper

In [45]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

In [46]:
from google.colab import userdata
searchapi_api_key = userdata.get('SEARCHAPI_API_KEY')

In [48]:
search = SearchApiAPIWrapper(searchapi_api_key=searchapi_api_key)

In [57]:
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
        output_format="json",
        return_direct=True,
    )
]

In [58]:
self_ask_with_search = initialize_agent(tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True)

In [59]:
output = self_ask_with_search("What is the GDP of US in dollars plus 6?")



> Entering new AgentExecutor chain...
 Yes.
Followup: What is the GDP of US in dollars?
Intermediate answer: $27.36 trillion


> Finished chain.


In [60]:
print(output)

{'input': 'What is the GDP of US in dollars plus 6?', 'output': '$27.36 trillion'}


In [62]:
self_ask_with_search("How many years old is Elon Musk?")



> Entering new AgentExecutor chain...
 No
So the final answer is: 49

> Finished chain.


{'input': 'How many years old is Elon Musk?', 'output': '49'}